# Comparing Performance
## seg_glm & ranger_03 on new data
## on a per-registrar basis

# Load Data, Models, Predictions & Lifts
tr_expiry, pf_expiry 

|           | train                   | test                   | **PREDS**             | **MODELS**                      | **LIFT_DF**          |
|-----------|-------------------------|------------------------|-----------------------|---------------------------------|----------------------|
| *seg_glm* | expiry_train_prepped_2_1| expiry_test_prepped_2_1| seg_glm_expiry_exptest| first_renewal_model_expiry_train| lift_df_seg_glm_expiry_exptest   
| *rf_500*  | expiry_train_df_sub     | expiry_test_df_sub | ranger_predict_03_expiry| ranger_03_expiry                | lift_df_ranger_03_expiry

In [1]:
# preds
seg_glm_expiry_exptest <- readRDS("/home/jupyter/local/Domains_202003/data/seg_glm_expiry_exptest")
load("/home/jupyter/local/Domains_202003/data/ranger_predict_03_expiry")

In [2]:
source('functions_eval.R')

source('load_prep_data_expiry.R')


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last




In [3]:
library(dplyr)
library(data.table)
library(partykit)
library(tictoc)
library(caret)
library(e1071)
library(randomForest)
library(ranger)

#for 3d plotting
library(akima)
library(plotly)

# for prep data
library(stringr)
library(pbapply)
library(stringdist)
library(data.table)

# for metrics
library(ROCR)

getwd()

Loading required package: grid

Loading required package: libcoin

Loading required package: mvtnorm

Loading required package: lattice

randomForest 4.6-14

Type rfNews() to see new features/changes/bug fixes.


Attaching package: ‘randomForest’


The following object is masked from ‘package:ggplot2’:

    margin


The following object is masked from ‘package:dplyr’:

    combine



Attaching package: ‘ranger’


The following object is masked from ‘package:randomForest’:

    importance



Attaching package: ‘plotly’


The following object is masked from ‘package:ggplot2’:

    last_plot


The following object is masked from ‘package:stats’:

    filter


The following object is masked from ‘package:graphics’:

    layout




[1] "/home/jupyter/local/Domains_202003/scripts/phaseII_gcloudaccess"

In [74]:
ls()

[1] "acc_0.5"                              
 [2] "auc"                                  
 [3] "auc_seg_glm_expiry_exptest"           
 [4] "auc_seg_glm_expiry_exptest_sub"       
 [5] "aucpr"                                
 [6] "calc_auc"                             
 [7] "chart_lift"                           
 [8] "cutoff"                               
 [9] "df"                                   
[10] "expiry_test_df_1"                     
[11] "expiry_test_df_sub"                   
[12] "expiry_test_prepped_2"                
[13] "expiry_test_prepped_2_1"              
[14] "expiry_train_df_1"                    
[15] "expiry_train_df_sub"                  
[16] "expiry_train_prepped_2"               
[17] "expiry_train_prepped_2_1"             
[18] "fnr_0.5"                              
[19] "fpr_0.5"                              
[20] "gains_plot_seg_glm_expiry_exptest"    
[21] "gains_plot_seg_glm_expiry_exptest_sub"
[22] "i"                                    
[23] "iterations"                           
[24] "joint_pred_df"                        
[25] "joint_pred_df_small"                  
[26] "joint_pred_df_sub"                    
[27] "lift_df_seg_glm_expiry_exptest"       
[28] "lift_df_seg_glm_expiry_exptest_sub"   
[29] "N"                                    
[30] "na_count"                             
[31] "obs_count"                            
[32] "output"                               
[33] "output2"                              
[34] "output3"                              
[35] "P"                                    
[36] "perf"                                 
[37] "plot_gains"                           
[38] "plot_multigains"                      
[39] "pred_obj"                             
[40] "ranger_03_perf_df"                    
[41] "ranger_predict_03_expiry"             
[42] "ranger_results"                       
[43] "reg"                                  
[44] "reg_name"                             
[45] "registrars"                           
[46] "seg_glm_expiry_exptest"               
[47] "seg_glm_expiry_exptest_pred_df"       
[48] "seg_glm_expiry_exptest_pred_df_sub"   
[49] "seg_glm_expiry_exptest_sub"           
[50] "seg_glm_perf_df"                      
[51] "seg_glm_perf_df_sub"                  
[52] "temp"                                 
[53] "TN"                                   
[54] "tnr_0.5"                              
[55] "TP"                                   
[56] "tpr_0.5"                              
[57] "variables"

In [5]:
options(repr.matrix.max.cols=50, repr.matrix.max.rows=100)

# seg_glm

In [6]:
# examine results
head(seg_glm_expiry_exptest)

renewal_type,renewed_count,expiry_date,domain_id,domain,creation_date,status,tld,registrar,reseller,reseller_country,region,reg_period,registrant_country,renewal_status,renew_mbg,renew_type,autorenew_type,renew_date,renew_registrar,renew_reseller,reg_revenue,reg_arpt,renew_period,renew_domain_revenue,renew_arpt,reg_arpt_org,tld_registrar_index,sld,sld_type,sld_length,sld_type2,day_domains,log_reg_arpt,gibb_score,pattern,cluster,pattern_score,pattern_domain_count,first_renewal_prediction
<chr>,<int>,<date>,<int>,<chr>,<date>,<chr>,<fct>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<fct>,<int>,<chr>,<chr>,<date>,<chr>,<chr>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<fct>,<int>,<chr>,<int>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<int>,<dbl>
FirstTime,1,2020-01-22,91639803,segeln.fun,2019-01-22,Active,fun,1&1 internet,1&1 internet,Germany,Non China,1,NA,Renewed,0,auto-renewal,realized,2020-01-22,1&1 Internet,1&1 Internet,15,15,1,15,15,15,fun1&1 internet,segeln,l,6,6l,1,2.708050,2.23,segeln,1,1,1,0.8459141
FirstTime,1,2020-03-07,95819759,harzhotel.fun,2019-03-07,Active,fun,1&1 internet,1&1 internet,Germany,Non China,1,NA,Renewed,0,auto-renewal,realized,2020-03-07,1&1 Internet,1&1 Internet,15,15,1,15,15,15,fun1&1 internet,harzhotel,l,9,9l,1,2.708050,3.45,harzhotel,1,1,1,0.4607455
FirstTime,1,2020-01-15,91044973,movistar.fun,2019-01-15,Active,fun,10dencehispahard,10dencehispahard,Spain,Non China,1,NA,Not Renewd,0,auto-renewal,unrealized,2020-01-15,10dencehispahard,10dencehispahard,15,15,1,15,15,15,fun10dencehispahard,movistar,l,8,8l,1,2.708050,8.62,movistar,1,1,1,NA
FirstTime,1,2020-02-01,92472913,kuinik.fun,2019-02-01,Active,fun,10dencehispahard,10dencehispahard,Spain,Non China,1,NA,Not Renewd,0,auto-renewal,unrealized,2020-02-01,10dencehispahard,10dencehispahard,15,15,1,15,15,15,fun10dencehispahard,kuinik,l,6,6l,1,2.708050,3.13,kuinik,1,1,1,NA
FirstTime,1,2020-01-21,91532433,bulgari.fun,2019-01-21,Active,fun,1api,1api,Germany,Non China,1,NA,Renewed,0,auto-renewal,realized,2020-01-21,1API,1API,5,5,1,15,15,5,fun1api,bulgari,l,7,7l,1,1.609438,4.51,bulgari,1,1,1,0.9999971
FirstTime,1,2020-04-24,101717674,gotsome.fun,2019-04-24,Active,fun,1api,1api,Germany,Non China,1,NA,Not Renewd,0,auto-renewal,unrealized,2020-04-24,1API,1API,5,5,1,15,15,5,fun1api,gotsome,l,7,7l,1,1.609438,6.09,gotsome,1,1,1,0.9999992


In [7]:
table(seg_glm_expiry_exptest$renewal_type)


FirstTime 
   296797 

In [8]:
iterations = length(unique(seg_glm_expiry_exptest$registrar))
variables = 14

output <- matrix(ncol=variables, nrow=iterations)

for(i in 1:iterations){
    reg = unique(seg_glm_expiry_exptest$registrar)[i]
    temp = subset(seg_glm_expiry_exptest, registrar==reg)
    obs_count = dim(temp)[1]
    na_count = sum(is.na(temp$first_renewal_prediction)) # na_count
    
    if (obs_count>na_count) {
        temp = temp[!is.na(temp$first_renewal_prediction)] 
        temp$Y = factor(as.numeric(temp$renewal_status == "Renewed"), levels=c(0,1))
        temp$pred = as.numeric(temp$first_renewal_prediction>0.5)
        
        TP = sum((temp$pred==1) & (temp$Y==1))
        TN = sum((temp$pred==0) & (temp$Y==0))
        P = sum((temp$Y==1))
        N = sum((temp$Y==0))
        

        pred_obj = prediction(temp$first_renewal_prediction, temp$Y)

        perf = performance(pred_obj,'acc')
        acc_0.5 = performance(pred_obj,'acc')@y.values[[1]][max(which(performance(pred_obj,'acc')@x.values[[1]] >= 0.5))] # acc_0.5

        perf = performance(pred_obj,"tpr")
        tpr_0.5 = perf@y.values[[1]][max(which(perf@x.values[[1]] >= 0.5))] # tpr_0.5 

        perf = performance(pred_obj,"tnr")
        tnr_0.5 = perf@y.values[[1]][max(which(perf@x.values[[1]] >= 0.5))] # tnr_0.5 

        perf = performance(pred_obj,"fpr")
        fpr_0.5 = perf@y.values[[1]][max(which(perf@x.values[[1]] >= 0.5))] # fpr_0.5 

        perf = performance(pred_obj,"fnr")
        fnr_0.5 = perf@y.values[[1]][max(which(perf@x.values[[1]] >= 0.5))] # fnr_0.5 

        auc = NA
        aucpr = NA
        tryCatch(
        {auc = performance(pred_obj,"auc")@y.values[[1]]
        aucpr = performance(pred_obj,"aucpr")@y.values[[1]]},
    error = function(e){
        # (Optional)
        # Do this at the end before quitting the tryCatch structure...
    })
        
        output[i,] <- c(reg, obs_count, na_count, TP, TN, P, N, acc_0.5, tpr_0.5, tnr_0.5, fpr_0.5, fnr_0.5, auc, aucpr )
    } else {
        output[i,] <- c(reg, obs_count, na_count, rep (NA,11))
    } 
}

In [63]:
seg_glm_perf_df <- data.frame(output) # %>% drop_na()
names(seg_glm_perf_df) <- c('reg', 'obs_count', 'na_count', 'TP', 'TN','P', 'N', 'acc_0.5', 'tpr_0.5', 'tnr_0.5', 'fpr_0.5', 'fnr_0.5', 'auc', 'aucpr')
head(seg_glm_perf_df,50)

,reg,obs_count,na_count,TP,TN,P,N,acc_0.5,tpr_0.5,tnr_0.5,fpr_0.5,fnr_0.5,auc,aucpr
,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>
1,1&1 internet,2321,6,460,863,1000,1315,0.571490280777538,0.46,0.656273764258555,0.343726235741445,0.54,0.587819011406844,0.486851264976771
2,10dencehispahard,165,2,2,128,31,132,0.797546012269939,0.0645161290322581,0.96969696969697,0.0303030303030303,0.935483870967742,0.706500488758553,0.348974984750745
3,1api,190,8,56,45,79,103,0.554945054945055,0.708860759493671,0.436893203883495,0.563106796116505,0.291139240506329,0.571033550448569,0.46988104455093
4,ascio,647,8,89,300,274,365,0.608763693270736,0.324817518248175,0.821917808219178,0.178082191780822,0.675182481751825,0.642855714428557,0.558515190603268
5,alibaba,30567,1,2,29507,1045,29521,0.965419093109992,0.00191387559808612,0.999525761322448,0.000474238677551573,0.998086124401914,0.861912215924738,0.165704392123854
6,alibaba sg,268,1,0,250,17,250,0.936329588014981,0,1,0,1,0.601529411764706,0.135296667638456
7,automattic,355,0,132,54,191,164,0.523943661971831,0.691099476439791,0.329268292682927,0.670731707317073,0.308900523560209,0.503958625973694,0.522754001638221
8,bizcn,231,11,1,176,24,196,0.804545454545455,0.0416666666666667,0.897959183673469,0.102040816326531,0.958333333333333,0.598001700680272,0.156782441367867
9,beget,1373,2,0,1258,108,1263,0.917578409919767,0,0.996041171813143,0.00395882818685669,1,0.741385883111937,0.240106011231044


In [65]:
reg_name='bb online'
seg_glm_perf_df %>% filter(reg == reg_name) 

reg,obs_count,na_count,TP,TN,P,N,acc_0.5,tpr_0.5,tnr_0.5,fpr_0.5,fnr_0.5,auc,aucpr
<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>
bb online,3,3,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [10]:
seg_glm_perf_df %>% arrange(desc(aucpr)) %>% select(reg,obs_count,na_count,aucpr)  %>% head(30)

,reg,obs_count,na_count,aucpr
,<fct>,<fct>,<fct>,<fct>
1,nameshield,9,6,1
2,alpnames,3,1,1
3,netistrar,3,0,1
4,tecnocratica centro,3,1,1
5,22net inc,51,0,1
6,csc corporate,25,18,0.988095238095238
7,corehub,10,2,0.972321428571428
8,registrygate,18,4,0.899309718059718
9,paragon internet,47,0,0.878850443062443


In [11]:
subset(seg_glm_perf_df, reg=='go daddy')

,reg,obs_count,na_count,TP,TN,P,N,acc_0.5,tpr_0.5,tnr_0.5,fpr_0.5,fnr_0.5,auc,aucpr
,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>
32,go daddy,27537,0,385,23188,4035,23502,0.856048226023169,0.0954151177199504,0.986639434941707,0.0133605650582929,0.90458488228005,0.764926099252595,0.358132658377875


# Rerun without missing predictions, compare

In [12]:
dim(seg_glm_expiry_exptest)
seg_glm_expiry_exptest_sub <- seg_glm_expiry_exptest %>% drop_na(first_renewal_prediction)
dim(seg_glm_expiry_exptest_sub)
(dim(seg_glm_expiry_exptest)-dim(seg_glm_expiry_exptest_sub))/dim(seg_glm_expiry_exptest)

[1] 296797     40

[1] 295443     40

[1] 0.004562041 0.000000000

In [13]:
iterations = length(unique(seg_glm_expiry_exptest_sub$registrar))
variables = 14

output3 <- matrix(ncol=variables, nrow=iterations)

for(i in 1:iterations){
    reg = unique(seg_glm_expiry_exptest_sub$registrar)[i]
    temp = subset(seg_glm_expiry_exptest_sub, registrar==reg)
    obs_count = dim(temp)[1]
    na_count = sum(is.na(temp$first_renewal_prediction)) # na_count
    
    if (obs_count>na_count) {
        temp = temp[!is.na(temp$first_renewal_prediction)] 
        temp$Y = factor(as.numeric(temp$renewal_status == "Renewed"), levels=c(0,1))
        temp$pred = as.numeric(temp$first_renewal_prediction>0.5)
        
        TP = sum((temp$pred==1) & (temp$Y==1))
        TN = sum((temp$pred==0) & (temp$Y==0))
        P = sum((temp$Y==1))
        N = sum((temp$Y==0))
        

        pred_obj = prediction(temp$first_renewal_prediction, temp$Y)

        perf = performance(pred_obj,'acc')
        acc_0.5 = performance(pred_obj,'acc')@y.values[[1]][max(which(performance(pred_obj,'acc')@x.values[[1]] >= 0.5))] # acc_0.5

        perf = performance(pred_obj,"tpr")
        tpr_0.5 = perf@y.values[[1]][max(which(perf@x.values[[1]] >= 0.5))] # tpr_0.5 

        perf = performance(pred_obj,"tnr")
        tnr_0.5 = perf@y.values[[1]][max(which(perf@x.values[[1]] >= 0.5))] # tnr_0.5 

        perf = performance(pred_obj,"fpr")
        fpr_0.5 = perf@y.values[[1]][max(which(perf@x.values[[1]] >= 0.5))] # fpr_0.5 

        perf = performance(pred_obj,"fnr")
        fnr_0.5 = perf@y.values[[1]][max(which(perf@x.values[[1]] >= 0.5))] # fnr_0.5 

        auc = NA
        aucpr = NA
        tryCatch(
        {auc = performance(pred_obj,"auc")@y.values[[1]]
        aucpr = performance(pred_obj,"aucpr")@y.values[[1]]},
    error = function(e){
        # (Optional)
        # Do this at the end before quitting the tryCatch structure...
    })
        
        output3[i,] <- c(reg, obs_count, na_count, TP, TN, P, N, acc_0.5, tpr_0.5, tnr_0.5, fpr_0.5, fnr_0.5, auc, aucpr )
    } else {
        output3[i,] <- c(reg, obs_count, na_count, rep (NA,11))
    } 
}

In [14]:
seg_glm_perf_df_sub <- data.frame(output3) %>% drop_na()
names(seg_glm_perf_df_sub) <- c('reg', 'obs_count', 'na_count', 'TP', 'TN','P', 'N', 'acc_0.5', 'tpr_0.5', 'tnr_0.5', 'fpr_0.5', 'fnr_0.5', 'auc', 'aucpr')

In [15]:
seg_glm_perf_df_sub %>% arrange(desc(aucpr)) %>% select(reg,obs_count,na_count,aucpr)  %>% head(30)

,reg,obs_count,na_count,aucpr
,<fct>,<fct>,<fct>,<fct>
1,alpnames,2,0,1
2,netistrar,3,0,1
3,tecnocratica centro,2,0,1
4,22net inc,51,0,1
5,nameshield,3,0,1
6,csc corporate,7,0,0.988095238095238
7,corehub,8,0,0.972321428571428
8,registrygate,14,0,0.899309718059718
9,paragon internet,47,0,0.878850443062443


## examine lift, etc.

In [16]:
seg_glm_expiry_exptest_pred_df = data.frame("actual" = seg_glm_expiry_exptest$renewal_status,
          "predicted" = seg_glm_expiry_exptest$first_renewal_prediction)
lift_df_seg_glm_expiry_exptest <- chart_lift(pred_df = seg_glm_expiry_exptest_pred_df,
                        dep_var = "actual",
                        pred_var = "predicted")
gains_plot_seg_glm_expiry_exptest <- plot_gains (lift_df_seg_glm_expiry_exptest) 
auc_seg_glm_expiry_exptest<- calc_auc(lift_df_seg_glm_expiry_exptest)

In [17]:
seg_glm_expiry_exptest_pred_df_sub = data.frame("actual" = seg_glm_expiry_exptest_sub$renewal_status,
          "predicted" = seg_glm_expiry_exptest_sub$first_renewal_prediction)

lift_df_seg_glm_expiry_exptest_sub <- chart_lift(pred_df = seg_glm_expiry_exptest_pred_df_sub,
                        dep_var = "actual",
                        pred_var = "predicted")
gains_plot_seg_glm_expiry_exptest_sub <- plot_gains (lift_df_seg_glm_expiry_exptest_sub) 
auc_seg_glm_expiry_exptest_sub<- calc_auc(lift_df_seg_glm_expiry_exptest_sub)



In [18]:
auc_seg_glm_expiry_exptest
auc_seg_glm_expiry_exptest_sub

[1] 0.8165071

[1] 0.8233043

In [19]:
lift_df_seg_glm_expiry_exptest
lift_df_seg_glm_expiry_exptest_sub

P,actu_renwd2,gain,lift
<dbl>,<int>,<dbl>,<dbl>
0.1,12787,0.4708893,4.708893
0.2,18692,0.6883447,3.441723
0.3,21933,0.8076966,2.692322
0.4,24000,0.8838151,2.209538
0.5,25227,0.9290002,1.858000
0.6,25947,0.9555146,1.592524
0.7,26325,0.9694347,1.384907
0.8,26587,0.9790830,1.223854
0.9,26647,0.9812926,1.090325


P,actu_renwd2,gain,lift
<dbl>,<int>,<dbl>,<dbl>
0.1,12759,0.4744534,4.744534
0.2,18660,0.6938867,3.469433
0.3,21891,0.8140339,2.713446
0.4,23979,0.8916778,2.229195
0.5,25207,0.9373420,1.874684
0.6,25927,0.9641157,1.606860
0.7,26312,0.9784322,1.397760
0.8,26576,0.9882493,1.235312
0.9,26646,0.9908523,1.100947


# ranger_03

In [20]:
head(expiry_test_df_sub)

renewal_status,tld,registrar,reseller_country,region,reg_period,sld_type,sld_length,day_domains,gibb_score,pattern_domain_count,reg_arpt
<fct>,<fct>,<chr>,<chr>,<chr>,<int>,<fct>,<int>,<int>,<dbl>,<int>,<dbl>
Renewed,fun,1&1 internet,Germany,Non China,1,l,6,1,2.23,1,15
Renewed,fun,1&1 internet,Germany,Non China,1,l,9,1,3.45,1,15
Not Renewd,fun,10dencehispahard,Spain,Non China,1,l,8,1,8.62,1,15
Not Renewd,fun,10dencehispahard,Spain,Non China,1,l,6,1,3.13,1,15
Renewed,fun,1api,Germany,Non China,1,l,7,1,4.51,1,5
Not Renewd,fun,1api,Germany,Non China,1,l,7,1,6.09,1,5


In [21]:
table(expiry_test_df_1$renewal_type)


FirstTime 
   296797 

In [22]:
head(ranger_predict_03_expiry)

[1] 0.5085864 0.4922921 0.5950325 0.4578849 0.5419494 0.5397591

In [23]:
ranger_results <- expiry_test_df_sub
ranger_results$predicted <- ranger_predict_03_expiry

In [24]:
head(ranger_results)

renewal_status,tld,registrar,reseller_country,region,reg_period,sld_type,sld_length,day_domains,gibb_score,pattern_domain_count,reg_arpt,predicted
<fct>,<fct>,<chr>,<chr>,<chr>,<int>,<fct>,<int>,<int>,<dbl>,<int>,<dbl>,<dbl>
Renewed,fun,1&1 internet,Germany,Non China,1,l,6,1,2.23,1,15,0.5085864
Renewed,fun,1&1 internet,Germany,Non China,1,l,9,1,3.45,1,15,0.4922921
Not Renewd,fun,10dencehispahard,Spain,Non China,1,l,8,1,8.62,1,15,0.5950325
Not Renewd,fun,10dencehispahard,Spain,Non China,1,l,6,1,3.13,1,15,0.4578849
Renewed,fun,1api,Germany,Non China,1,l,7,1,4.51,1,5,0.5419494
Not Renewd,fun,1api,Germany,Non China,1,l,7,1,6.09,1,5,0.5397591


In [25]:
iterations = length(unique(ranger_results$registrar))
variables = 14

output2 <- matrix(ncol=variables, nrow=iterations)

for(i in 1:iterations){
    reg = unique(ranger_results$registrar)[i]
    temp = subset(ranger_results, registrar==reg)
    obs_count = dim(temp)[1]
    na_count = sum(is.na(temp$predicted)) # na_count
    
    if (obs_count>na_count) {
        temp = temp[!is.na(temp$predicted)] 
        temp$Y = factor(as.numeric(temp$renewal_status == "Renewed"), levels=c(0,1))
        temp$pred = as.numeric(temp$predicted>0.5)
        
        TP = sum((temp$pred==1) & (temp$Y==1))
        TN = sum((temp$pred==0) & (temp$Y==0))
        P = sum((temp$Y==1))
        N = sum((temp$Y==0))
        

        pred_obj = prediction(temp$predicted, temp$Y)

        perf = performance(pred_obj,'acc')
        acc_0.5 = performance(pred_obj,'acc')@y.values[[1]][max(which(performance(pred_obj,'acc')@x.values[[1]] >= 0.5))] # acc_0.5

        perf = performance(pred_obj,"tpr")
        tpr_0.5 = perf@y.values[[1]][max(which(perf@x.values[[1]] >= 0.5))] # tpr_0.5 

        perf = performance(pred_obj,"tnr")
        tnr_0.5 = perf@y.values[[1]][max(which(perf@x.values[[1]] >= 0.5))] # tnr_0.5 

        perf = performance(pred_obj,"fpr")
        fpr_0.5 = perf@y.values[[1]][max(which(perf@x.values[[1]] >= 0.5))] # fpr_0.5 

        perf = performance(pred_obj,"fnr")
        fnr_0.5 = perf@y.values[[1]][max(which(perf@x.values[[1]] >= 0.5))] # fnr_0.5 

        auc = NA
        aucpr = NA
        tryCatch(
        {auc = performance(pred_obj,"auc")@y.values[[1]]
        aucpr = performance(pred_obj,"aucpr")@y.values[[1]]},
    error = function(e){
        # (Optional)
        # Do this at the end before quitting the tryCatch structure...
    })
        
        output2[i,] <- c(reg, obs_count, na_count, TP, TN, P, N, acc_0.5, tpr_0.5, tnr_0.5, fpr_0.5, fnr_0.5, auc, aucpr )
    } else {
        output2[i,] <- c(reg, obs_count, na_count, rep (NA,11))
    } 
}

In [26]:
dim(output2)

[1] 155  14

In [27]:
output2

1&1 internet,2321,0,231,1090,1003,1318,0.569151227919,0.230309072781655,0.82701062215478,0.17298937784522,0.769690927218345,0.565424364236577,0.484720299622771
10dencehispahard,165,0,1,132,31,134,0.806060606060606,0.032258064516129,0.985074626865672,0.0149253731343284,0.967741935483871,0.708714492055849,0.328616915612339
1api,190,0,21,70,84,106,0.478947368421053,0.25,0.660377358490566,0.339622641509434,0.75,0.497753818508536,0.462086089935954
ascio,647,0,52,340,279,368,0.605873261205564,0.186379928315412,0.923913043478261,0.0760869565217391,0.813620071684588,0.602364812217547,0.543489670646984
alibaba,30567,0,4,29513,1045,29522,0.965649229561292,0.00382775119617225,0.999695142605515,0.000304857394485468,0.996172248803828,0.875427732266171,0.172572934737246
alibaba sg,268,0,0,251,17,251,0.936567164179104,0,1,0,1,0.613545816733067,0.180529008087729
automattic,355,0,187,15,191,164,0.569014084507042,0.979057591623037,0.0914634146341463,0.908536585365854,0.0209424083769634,0.547599284893373,0.55669670916807
bizcn,231,0,0,204,26,205,0.883116883116883,0,0.995121951219512,0.0048780487804878,1,0.670075046904316,0.219955173736057
beget,1373,0,0,1264,108,1265,0.920611798980335,0,0.999209486166008,0.000790513833992095,1,0.726709120187374,0.2068979169074
beijing guoxu,2,0,0,2,0,2,1,NaN,1,0,NaN,NA,NA
pdr ltd,16402,0,36,14292,2084,14318,0.873552005852945,0.017274472168906,0.998184103925129,0.00181589607487079,0.982725527831094,0.728867888131368,0.271207128958128


In [66]:
ranger_03_perf_df <- data.frame(output2) #%>% drop_na()
names(ranger_03_perf_df) <- c('reg', 'obs_count', 'na_count', 'TP', 'TN','P', 'N', 'acc_0.5', 'tpr_0.5', 'tnr_0.5', 'fpr_0.5', 'fnr_0.5', 'auc', 'aucpr')

In [67]:
reg_name='bb online'
ranger_03_perf_df %>% filter(reg == reg_name) 

reg,obs_count,na_count,TP,TN,P,N,acc_0.5,tpr_0.5,tnr_0.5,fpr_0.5,fnr_0.5,auc,aucpr
<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>
bb online,3,0,3,0,3,0,1,1,NaN,NaN,0,NA,NA


In [29]:
ranger_03_perf_df %>% arrange(desc(aucpr)) %>% select(reg,obs_count,na_count,aucpr)  %>% head(30)

,reg,obs_count,na_count,aucpr
,<fct>,<fct>,<fct>,<fct>
1,ejee group,8,0,1
2,epik llc,54,0,1
3,nameshield,9,0,1
4,dnc holdings,2,0,1
5,alpnames,3,0,1
6,blacknight,4,0,1
7,com laude,8,0,1
8,corehub,10,0,1
9,domaindiscover.com,4,0,1


In [30]:
subset(ranger_03_perf_df, reg=='go daddy')

,reg,obs_count,na_count,TP,TN,P,N,acc_0.5,tpr_0.5,tnr_0.5,fpr_0.5,fnr_0.5,auc,aucpr
,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>
33,go daddy,27537,0,266,23289,4035,23502,0.855394560046483,0.0659231722428748,0.99093694153689,0.00906305846310952,0.934076827757125,0.690613944427409,0.336712679608358


# Combine two performance dfs

In [31]:
registrars = data.frame(unique(ranger_results$registrar))
names(registrars) <- "reg"
dim(registrars)
head(registrars)

[1] 155   1

,reg
,<fct>
1,1&1 internet
2,10dencehispahard
3,1api
4,ascio
5,alibaba
6,alibaba sg


In [68]:
dim(ranger_03_perf_df)
head(ranger_03_perf_df)

[1] 155  14

,reg,obs_count,na_count,TP,TN,P,N,acc_0.5,tpr_0.5,tnr_0.5,fpr_0.5,fnr_0.5,auc,aucpr
,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>
1,1&1 internet,2321,0,231,1090,1003,1318,0.569151227919,0.230309072781655,0.82701062215478,0.17298937784522,0.769690927218345,0.565424364236577,0.484720299622771
2,10dencehispahard,165,0,1,132,31,134,0.806060606060606,0.032258064516129,0.985074626865672,0.0149253731343284,0.967741935483871,0.708714492055849,0.328616915612339
3,1api,190,0,21,70,84,106,0.478947368421053,0.25,0.660377358490566,0.339622641509434,0.75,0.497753818508536,0.462086089935954
4,ascio,647,0,52,340,279,368,0.605873261205564,0.186379928315412,0.923913043478261,0.0760869565217391,0.813620071684588,0.602364812217547,0.543489670646984
5,alibaba,30567,0,4,29513,1045,29522,0.965649229561292,0.00382775119617225,0.999695142605515,0.000304857394485468,0.996172248803828,0.875427732266171,0.172572934737246
6,alibaba sg,268,0,0,251,17,251,0.936567164179104,0,1,0,1,0.613545816733067,0.180529008087729


In [69]:
dim(seg_glm_perf_df)
head(seg_glm_perf_df)

[1] 155  14

,reg,obs_count,na_count,TP,TN,P,N,acc_0.5,tpr_0.5,tnr_0.5,fpr_0.5,fnr_0.5,auc,aucpr
,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>
1,1&1 internet,2321,6,460,863,1000,1315,0.571490280777538,0.46,0.656273764258555,0.343726235741445,0.54,0.587819011406844,0.486851264976771
2,10dencehispahard,165,2,2,128,31,132,0.797546012269939,0.0645161290322581,0.96969696969697,0.0303030303030303,0.935483870967742,0.706500488758553,0.348974984750745
3,1api,190,8,56,45,79,103,0.554945054945055,0.708860759493671,0.436893203883495,0.563106796116505,0.291139240506329,0.571033550448569,0.46988104455093
4,ascio,647,8,89,300,274,365,0.608763693270736,0.324817518248175,0.821917808219178,0.178082191780822,0.675182481751825,0.642855714428557,0.558515190603268
5,alibaba,30567,1,2,29507,1045,29521,0.965419093109992,0.00191387559808612,0.999525761322448,0.000474238677551573,0.998086124401914,0.861912215924738,0.165704392123854
6,alibaba sg,268,1,0,250,17,250,0.936329588014981,0,1,0,1,0.601529411764706,0.135296667638456


In [70]:
joint_pred_df <- merge(registrars, merge(seg_glm_perf_df, ranger_03_perf_df, 
                       by = 'reg',
                       sort = TRUE, 
                       suffixes = c("_s","_r")) %>% mutate_at(c('obs_count_s','na_count_s','TP_s','TN_s','P_s','N_s','acc_0.5_s','tpr_0.5_s','tnr_0.5_s',
                                                              'fpr_0.5_s','fnr_0.5_s','auc_s','aucpr_s','obs_count_r','na_count_r','TP_r','TN_r','P_r','N_r',
                                                              'acc_0.5_r','tpr_0.5_r','tnr_0.5_r','fpr_0.5_r','fnr_0.5_r','auc_r','aucpr_r'),
                                                              as.character) %>% mutate_at(c('obs_count_s','na_count_s','TP_s','TN_s','P_s','N_s','acc_0.5_s','tpr_0.5_s','tnr_0.5_s',
                                                              'fpr_0.5_s','fnr_0.5_s','auc_s','aucpr_s','obs_count_r','na_count_r','TP_r','TN_r','P_r','N_r',
                                                              'acc_0.5_r','tpr_0.5_r','tnr_0.5_r','fpr_0.5_r','fnr_0.5_r','auc_r','aucpr_r'),
                                                              as.numeric) , by = 'reg', all.x=TRUE)
dim(joint_pred_df)
head(joint_pred_df)

[1] 155  27

,reg,obs_count_s,na_count_s,TP_s,TN_s,P_s,N_s,acc_0.5_s,tpr_0.5_s,tnr_0.5_s,fpr_0.5_s,fnr_0.5_s,auc_s,aucpr_s,obs_count_r,na_count_r,TP_r,TN_r,P_r,N_r,acc_0.5_r,tpr_0.5_r,tnr_0.5_r,fpr_0.5_r,fnr_0.5_r,auc_r,aucpr_r
,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1&1 internet,2321,6,460,863,1000,1315,0.5714903,0.46000000,0.6562738,0.34372624,0.5400000,0.5878190,0.4868513,2321,0,231,1090,1003,1318,0.5691512,0.23030907,0.8270106,0.17298938,0.7696909,0.5654244,0.4847203
2,101 domain,32,6,14,6,16,10,0.7692308,0.87500000,0.6000000,0.40000000,0.1250000,0.6656250,0.6992856,32,0,19,2,19,13,0.6562500,1.00000000,0.1538462,0.84615385,0.0000000,0.6923077,0.8021654
3,10dencehispahard,165,2,2,128,31,132,0.7975460,0.06451613,0.9696970,0.03030303,0.9354839,0.7065005,0.3489750,165,0,1,132,31,134,0.8060606,0.03225806,0.9850746,0.01492537,0.9677419,0.7087145,0.3286169
4,1api,190,8,56,45,79,103,0.5549451,0.70886076,0.4368932,0.56310680,0.2911392,0.5710336,0.4698810,190,0,21,70,84,106,0.4789474,0.25000000,0.6603774,0.33962264,0.7500000,0.4977538,0.4620861
5,22net inc,51,0,2,49,2,49,1.0000000,1.00000000,1.0000000,0.00000000,0.0000000,1.0000000,1.0000000,51,0,2,49,2,49,1.0000000,1.00000000,1.0000000,0.00000000,0.0000000,1.0000000,1.0000000
6,ab name isp,15,3,7,0,7,5,0.5833333,1.00000000,0.0000000,1.00000000,0.0000000,0.5571429,0.7285714,15,0,7,1,9,6,0.5333333,0.77777778,0.1666667,0.83333333,0.2222222,0.3888889,0.5754147


In [71]:
reg_name='bb online'
joint_pred_df %>% filter(reg == reg_name) 

reg,obs_count_s,na_count_s,TP_s,TN_s,P_s,N_s,acc_0.5_s,tpr_0.5_s,tnr_0.5_s,fpr_0.5_s,fnr_0.5_s,auc_s,aucpr_s,obs_count_r,na_count_r,TP_r,TN_r,P_r,N_r,acc_0.5_r,tpr_0.5_r,tnr_0.5_r,fpr_0.5_r,fnr_0.5_r,auc_r,aucpr_r
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
bb online,3,3,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,3,0,3,0,3,0,1,1,NaN,NaN,0,NA,NA


In [72]:
joint_pred_df %>% summarise_all(funs(sum(is.na(.))))

reg,obs_count_s,na_count_s,TP_s,TN_s,P_s,N_s,acc_0.5_s,tpr_0.5_s,tnr_0.5_s,fpr_0.5_s,fnr_0.5_s,auc_s,aucpr_s,obs_count_r,na_count_r,TP_r,TN_r,P_r,N_r,acc_0.5_r,tpr_0.5_r,tnr_0.5_r,fpr_0.5_r,fnr_0.5_r,auc_r,aucpr_r
<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
0,0,0,27,27,27,27,27,39,32,32,39,44,44,0,0,0,0,0,0,0,12,22,22,12,34,34


In [73]:
joint_pred_df %>% filter(is.na(obs_count_s))

reg,obs_count_s,na_count_s,TP_s,TN_s,P_s,N_s,acc_0.5_s,tpr_0.5_s,tnr_0.5_s,fpr_0.5_s,fnr_0.5_s,auc_s,aucpr_s,obs_count_r,na_count_r,TP_r,TN_r,P_r,N_r,acc_0.5_r,tpr_0.5_r,tnr_0.5_r,fpr_0.5_r,fnr_0.5_r,auc_r,aucpr_r
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>


In [75]:
# compare AUCpr
joint_pred_df$aucpr_srrat = joint_pred_df$aucpr_s/joint_pred_df$aucpr_r
joint_pred_df[c("reg","obs_count_s","na_count_s","aucpr_s","aucpr_r","aucpr_srrat")] %>% arrange(desc(aucpr_srrat))

reg,obs_count_s,na_count_s,aucpr_s,aucpr_r,aucpr_srrat
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
gmo,31494,8,0.3860741,0.11609370,3.325539
nhan hoa software,114,8,0.5610501,0.22609649,2.481463
p.a viet nam,258,8,0.2551691,0.11877589,2.148324
web4africa,29,5,0.5208333,0.26322751,1.978643
dotserve inc,1350,0,0.5536404,0.28976421,1.910658
ru-center,1196,5,0.2853623,0.18659874,1.529283
epag,42,0,0.4148106,0.27256335,1.521887
pdr ltd,16402,473,0.4027712,0.27120713,1.485105
dynadot,4481,2,0.1668540,0.11373646,1.467023


In [76]:
df = joint_pred_df_sub

cat("seg_glm outperforms ranger_03 on",nrow(df[df$aucpr_srrat>1,]),"registrars: ",nrow(df[df$aucpr_srrat>1,])/nrow(df),"\n") 
cat("ranger_03 outperforms seg_glm on",nrow(df[df$aucpr_srrat<1,]),"registrars: ",nrow(df[df$aucpr_srrat<1,])/nrow(df),"\n\n") 

cat("seg_glm outperforms ranger_03 on",sum(df[df$aucpr_srrat>1,"obs_count_s"]),"domains: ",sum(df[df$aucpr_srrat>1,"obs_count_s"])/sum(df["obs_count_s"]),"\n") 
cat("ranger_03 outperforms seg_glm on",sum(df[df$aucpr_srrat<1,"obs_count_s"]),"domains: ",sum(df[df$aucpr_srrat<1,"obs_count_s"])/sum(df["obs_count_s"]),"\n\n") 

seg_glm outperforms ranger_03 on 58 registrars:  0.5225225 
ranger_03 outperforms seg_glm on 47 registrars:  0.4234234 

seg_glm outperforms ranger_03 on 218975 domains:  0.7385628 
ranger_03 outperforms seg_glm on 77438 domains:  0.2611843 



In [77]:
cutoff = 600
joint_pred_df_small  <- joint_pred_df_sub[joint_pred_df_sub$obs_count_s<cutoff,]

cat("On the ",nrow(joint_pred_df_small)," registrars w/ less than ",cutoff," domains renewing....\n\n")
cat("seg_glm outperforms ranger_03 on",nrow(joint_pred_df_small[joint_pred_df_small$aucpr_srrat>1,]),"registrars: ",nrow(joint_pred_df_small[joint_pred_df_small$aucpr_srrat>1,])/nrow(joint_pred_df_small),"\n") 
cat("ranger_03 outperforms seg_glm on",nrow(joint_pred_df_small[joint_pred_df_small$aucpr_srrat<1,]),"registrars: ",nrow(joint_pred_df_small[joint_pred_df_small$aucpr_srrat<1,])/nrow(joint_pred_df_small),"\n\n") 

cat("seg_glm outperforms ranger_03 on",sum(joint_pred_df_small[joint_pred_df_small$aucpr_srrat>1,"obs_count_s"]),"domains: ",sum(joint_pred_df_small[joint_pred_df_small$aucpr_srrat>1,"obs_count_s"])/sum(joint_pred_df_small["obs_count_s"]),"\n") 
cat("ranger_03 outperforms seg_glm on",sum(joint_pred_df_small[joint_pred_df_small$aucpr_srrat<1,"obs_count_s"]),"domains: ",sum(joint_pred_df_small[joint_pred_df_small$aucpr_srrat<1,"obs_count_s"])/sum(joint_pred_df_small["obs_count_s"]),"\n\n") 

On the  70  registrars w/ less than  600  domains renewing....

seg_glm outperforms ranger_03 on 29 registrars:  0.4142857 
ranger_03 outperforms seg_glm on 35 registrars:  0.5 

seg_glm outperforms ranger_03 on 3791 domains:  0.446315 
ranger_03 outperforms seg_glm on 4628 domains:  0.5448552 



In [78]:
joint_pred_df_small

,reg,obs_count_s,na_count_s,TP_s,TN_s,P_s,N_s,acc_0.5_s,tpr_0.5_s,tnr_0.5_s,fpr_0.5_s,fnr_0.5_s,auc_s,aucpr_s,obs_count_r,na_count_r,TP_r,TN_r,P_r,N_r,acc_0.5_r,tpr_0.5_r,tnr_0.5_r,fpr_0.5_r,fnr_0.5_r,auc_r,aucpr_r,aucpr_srrat
,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2,101 domain,32,6,14,6,16,10,0.7692308,0.87500000,0.6000000,0.400000000,0.12500000,0.6656250,0.69928561,32,0,19,2,19,13,0.6562500,1.00000000,0.15384615,0.846153846,0.00000000,0.6923077,0.8021654,0.8717474
3,10dencehispahard,165,2,2,128,31,132,0.7975460,0.06451613,0.9696970,0.030303030,0.93548387,0.7065005,0.34897498,165,0,1,132,31,134,0.8060606,0.03225806,0.98507463,0.014925373,0.96774194,0.7087145,0.3286169,1.0619508
4,1api,190,8,56,45,79,103,0.5549451,0.70886076,0.4368932,0.563106796,0.29113924,0.5710336,0.46988104,190,0,21,70,84,106,0.4789474,0.25000000,0.66037736,0.339622642,0.75000000,0.4977538,0.4620861,1.0168691
5,22net inc,51,0,2,49,2,49,1.0000000,1.00000000,1.0000000,0.000000000,0.00000000,1.0000000,1.00000000,51,0,2,49,2,49,1.0000000,1.00000000,1.00000000,0.000000000,0.00000000,1.0000000,1.0000000,1.0000000
6,ab name isp,15,3,7,0,7,5,0.5833333,1.00000000,0.0000000,1.000000000,0.00000000,0.5571429,0.72857143,15,0,7,1,9,6,0.5333333,0.77777778,0.16666667,0.833333333,0.22222222,0.3888889,0.5754147,1.2661675
8,alibaba sg,268,1,0,250,17,250,0.9363296,0.00000000,1.0000000,0.000000000,1.00000000,0.6015294,0.13529667,268,0,0,251,17,251,0.9365672,0.00000000,1.00000000,0.000000000,1.00000000,0.6135458,0.1805290,0.7494456
9,alpnames,3,1,0,1,1,1,0.5000000,0.00000000,1.0000000,0.000000000,1.00000000,1.0000000,1.00000000,3,0,1,2,1,2,1.0000000,1.00000000,1.00000000,0.000000000,0.00000000,1.0000000,1.0000000,1.0000000
11,atak,31,1,0,25,4,26,0.8333333,0.00000000,0.9615385,0.038461538,1.00000000,0.4423077,0.25393908,31,0,0,25,5,26,0.8064516,0.00000000,0.96153846,0.038461538,1.00000000,0.7769231,0.4261905,0.5958347
12,automattic,355,0,132,54,191,164,0.5239437,0.69109948,0.3292683,0.670731707,0.30890052,0.5039586,0.52275400,355,0,187,15,191,164,0.5690141,0.97905759,0.09146341,0.908536585,0.02094241,0.5475993,0.5566967,0.9390284


In [ ]:
# compare acc_0.5
# replaced NA's and 0s with very small values for ratio calc
joint_pred_df <- joint_pred_df %>%  
                    mutate_at(vars(acc_0.5_s, acc_0.5_r), ~replace_na(., 0.000001)) %>%
                    mutate_at(vars(acc_0.5_s, acc_0.5_r), ~replace(., .==0, 0.000001)) 
joint_pred_df$acc_srrat = joint_pred_df$acc_0.5_s/joint_pred_df$acc_0.5_r
joint_pred_df[c("reg","obs_count_s","na_count_s","acc_0.5_s","acc_0.5_r","acc_srrat")] %>% arrange(desc(acc_srrat))

In [82]:
df = joint_pred_df

cat("seg_glm outperforms ranger_03 on",nrow(df[df$acc_srrat>1,]),"registrars: ",nrow(df[df$acc_srrat>1,])/nrow(df),"\n") 
cat("ranger_03 outperforms seg_glm on",nrow(df[df$acc_srrat<1,]),"registrars: ",nrow(df[df$acc_srrat<1,])/nrow(df),"\n\n") 

cat("seg_glm outperforms ranger_03 on",sum(df[df$acc_srrat>1,"obs_count_s"]),"domains: ",sum(df[df$acc_srrat>1,"obs_count_s"])/sum(df["obs_count_s"]),"\n") 
cat("ranger_03 outperforms seg_glm on",sum(df[df$acc_srrat<1,"obs_count_s"]),"domains: ",sum(df[df$acc_srrat<1,"obs_count_s"])/sum(df["obs_count_s"]),"\n\n") 

seg_glm outperforms ranger_03 on 51 registrars:  0.3290323 
ranger_03 outperforms seg_glm on 86 registrars:  0.5548387 

seg_glm outperforms ranger_03 on 152087 domains:  0.5124277 
ranger_03 outperforms seg_glm on 133261 domains:  0.4489971 



In [84]:
cutoff = 600
joint_pred_df_small  <- joint_pred_df[joint_pred_df$obs_count_s<cutoff,]

cat("On the ",nrow(joint_pred_df_small)," registrars w/ less than ",cutoff," domains renewing....\n\n")
cat("seg_glm outperforms ranger_03 on",nrow(joint_pred_df_small[joint_pred_df_small$acc_srrat>1,]),"registrars: ",nrow(joint_pred_df_small[joint_pred_df_small$acc_srrat>1,])/nrow(joint_pred_df_small),"\n") 
cat("ranger_03 outperforms seg_glm on",nrow(joint_pred_df_small[joint_pred_df_small$acc_srrat<1,]),"registrars: ",nrow(joint_pred_df_small[joint_pred_df_small$acc_srrat<1,])/nrow(joint_pred_df_small),"\n\n") 

cat("seg_glm outperforms ranger_03 on",sum(joint_pred_df_small[joint_pred_df_small$acc_srrat>1,"obs_count_s"]),"domains: ",sum(joint_pred_df_small[joint_pred_df_small$acc_srrat>1,"obs_count_s"])/sum(joint_pred_df_small["obs_count_s"]),"\n") 
cat("ranger_03 outperforms seg_glm on",sum(joint_pred_df_small[joint_pred_df_small$acc_srrat<1,"obs_count_s"]),"domains: ",sum(joint_pred_df_small[joint_pred_df_small$acc_srrat<1,"obs_count_s"])/sum(joint_pred_df_small["obs_count_s"]),"\n\n") 

On the  114  registrars w/ less than  600  domains renewing....

seg_glm outperforms ranger_03 on 28 registrars:  0.245614 
ranger_03 outperforms seg_glm on 69 registrars:  0.6052632 

seg_glm outperforms ranger_03 on 3009 domains:  0.3418153 
ranger_03 outperforms seg_glm on 5485 domains:  0.623083 



In [86]:
write.csv(joint_pred_df, "../../data/output/joint_pred_df.csv", row.names=FALSE)

In [90]:
system("gsutil cp /home/jupyter/local/Domains_202003/data/output/* gs://data_outputt/output/ ")
system("mv /home/jupyter/local/Domains_202003/data/output/* /home/jupyter/local/Domains_202003/data/")